
# KDD Cup 1999 - Anomali Tespiti

Bu çalışma, KDD Cup 1999 veri seti üzerinde **anomali tespiti (intrusion detection)** için 
klasik makine öğrenmesi algoritmaları ile model geliştirme sürecini içermektedir.


In [ ]:

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    roc_auc_score, roc_curve, precision_recall_curve,
    average_precision_score, f1_score, confusion_matrix,
    classification_report
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import joblib

plt.rcParams['figure.figsize'] = (8, 6)



## I. Veri Setinin Yüklenmesi

Bu bölümde **KDD Cup 1999** veri seti `pandas` ile okunarak DataFrame'e yüklenir.  
Veri seti, ağ trafiğinden elde edilen **41 özellik** ve bir etiket sütunu (`label`) içerir.  
Etiketler "normal" veya farklı saldırı türleridir.


In [ ]:

# Veri yolu
DATA_PATH = "kdd99_10percent.csv"  # CSV dosyasını proje klasörüne koyunuz

# Kolon isimleri
KDD_COLUMNS = [
    "duration","protocol_type","service","flag","src_bytes","dst_bytes","land",
    "wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised",
    "root_shell","su_attempted","num_root","num_file_creations","num_shells",
    "num_access_files","num_outbound_cmds","is_host_login","is_guest_login",
    "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",
    "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count",
    "dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate",
    "dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
    "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate",
    "label"
]

df = pd.read_csv(DATA_PATH, header=None, names=KDD_COLUMNS)

# İkili sınıf (Normal=0, Attack=1) dönüşümü
df['label_binary'] = np.where(df['label'].astype(str).str.contains('normal'), 0, 1)

print("Boyut:", df.shape)
print("Saldırı oranı (1):", round(df['label_binary'].mean(), 4))
df.head()



## II. Veri Ön İşleme

Bu bölümde:
- Kategorik değişkenler One-Hot Encoding yöntemi ile dönüştürülür.  
- Sayısal değişkenler StandardScaler ile ölçeklendirilir.  
- Veri %80 eğitim, %20 test olarak ayrılır.


In [ ]:

categorical_cols = ['protocol_type', 'service', 'flag']
numeric_cols = [c for c in df.columns if c not in categorical_cols + ['label', 'label_binary']]

X = df[categorical_cols + numeric_cols]
y = df['label_binary']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(with_mean=False), numeric_cols)
    ]
)



## III. Model Kurulumu

Bu bölümde iki farklı klasik makine öğrenmesi algoritması uygulanacaktır:
- **Lojistik Regresyon** (baseline, açıklanabilir)  
- **Random Forest** (güçlü, özellik önemleri çıkarılabilir)


In [ ]:

logreg_pipe = Pipeline([
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=1000, random_state=42))
])

rf_pipe = Pipeline([
    ("prep", preprocess),
    ("clf", RandomForestClassifier(random_state=42, n_jobs=-1))
])



## IV. Hiperparametre Optimizasyonu

Bu bölümde GridSearchCV kullanılarak en uygun hiperparametreler seçilecektir.


In [ ]:

logreg_param_grid = {
    "clf__C": [0.1, 1.0, 3.0],
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs", "liblinear"]
}

rf_param_grid = {
    "clf__n_estimators": [100, 200],
    "clf__max_depth": [None, 20, 40],
    "clf__min_samples_split": [2, 5],
    "clf__min_samples_leaf": [1, 2]
}

logreg_gs = GridSearchCV(logreg_pipe, logreg_param_grid, scoring="roc_auc", cv=3, n_jobs=-1, verbose=1)
rf_gs = GridSearchCV(rf_pipe, rf_param_grid, scoring="roc_auc", cv=3, n_jobs=-1, verbose=1)

print(">> Lojistik Regresyon GridSearch başlıyor...")
logreg_gs.fit(X_train, y_train)

print(">> Random Forest GridSearch başlıyor...")
rf_gs.fit(X_train, y_train)

print("LogReg En iyi:", logreg_gs.best_params_)
print("RF En iyi:", rf_gs.best_params_)



## V. Model Değerlendirme

Bu bölümde her iki model aşağıdaki metriklerle değerlendirilir:  
- ROC-AUC  
- Precision-Recall Eğrisi  
- F1 Skoru  
- Confusion Matrix  


In [ ]:

def evaluate_classifier(name, model, X_tr, y_tr, X_te, y_te):
    y_tr_proba = model.predict_proba(X_tr)[:, 1]
    y_te_proba = model.predict_proba(X_te)[:, 1]

    y_tr_pred = (y_tr_proba >= 0.5).astype(int)
    y_te_pred = (y_te_proba >= 0.5).astype(int)

    print(f"\n[{name}]")
    print("ROC-AUC  (train/test):", round(roc_auc_score(y_tr, y_tr_proba),4), "/", round(roc_auc_score(y_te, y_te_proba),4))
    print("F1-score (test):", round(f1_score(y_te, y_te_pred),4))
    print("\nClassification Report (Test):\n", classification_report(y_te, y_te_pred, digits=4))

    # ROC Eğrisi
    fpr, tpr, _ = roc_curve(y_te, y_te_proba)
    plt.figure()
    plt.plot(fpr, tpr, label=f"{name}")
    plt.plot([0,1],[0,1],'--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Eğrisi")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Precision-Recall Eğrisi
    prec, rec, _ = precision_recall_curve(y_te, y_te_proba)
    plt.figure()
    plt.plot(rec, prec, label=f"{name}")
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Eğrisi")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # Confusion Matrix
    cm = confusion_matrix(y_te, y_te_pred)
    plt.figure()
    plt.imshow(cm, interpolation='nearest')
    plt.title(f'Confusion Matrix - {name}')
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['Normal (0)', 'Attack (1)'])
    plt.yticks(tick_marks, ['Normal (0)', 'Attack (1)'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, cm[i, j], ha='center', va='center')
    plt.tight_layout()
    plt.show()


In [ ]:

best_logreg = logreg_gs.best_estimator_
best_rf = rf_gs.best_estimator_

evaluate_classifier("Logistic Regression", best_logreg, X_train, y_train, X_test, y_test)
evaluate_classifier("Random Forest", best_rf, X_train, y_train, X_test, y_test)



## VI. Sonuçlar ve Tartışma

Bu bölümde model sonuçları karşılaştırılır, Random Forest için **özellik önemleri** görselleştirilir ve 
istenirse **eşik ayarı** yapılarak precision/recall dengesi optimize edilir.


In [ ]:

# Özellik önemleri (RF)
ohe = best_rf.named_steps['prep'].named_transformers_['cat']
cat_feature_names = list(ohe.get_feature_names_out(['protocol_type','service','flag']))
num_feature_names = [c for c in df.columns if c not in ['protocol_type','service','flag','label','label_binary']]
all_feature_names = cat_feature_names + num_feature_names

rf = best_rf.named_steps['clf']
importances = rf.feature_importances_

feat_imp = pd.DataFrame({
    "feature": all_feature_names,
    "importance": importances
}).sort_values("importance", ascending=False).head(20)

display(feat_imp)

plt.figure()
plt.barh(feat_imp['feature'][::-1], feat_imp['importance'][::-1])
plt.xlabel("Importance")
plt.title("Top 20 Feature Importances (Random Forest)")
plt.tight_layout()
plt.show()

# Opsiyonel: Eşik ayarı (RF)
y_proba = best_rf.predict_proba(X_test)[:, 1]
thresholds = np.linspace(0.1, 0.9, 9)
rows = []
for th in thresholds:
    y_pred = (y_proba >= th).astype(int)
    tp = ((y_pred == 1) & (y_test == 1)).sum()
    fp = ((y_pred == 1) & (y_test == 0)).sum()
    fn = ((y_pred == 0) & (y_test == 1)).sum()
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
    rows.append({"threshold": th, "precision": precision, "recall": recall, "f1": f1_score(y_test, y_pred)})
th_df = pd.DataFrame(rows).sort_values("f1", ascending=False)
display(th_df)



## VII. Modelin Kaydedilmesi

Eğitilen en iyi modeller ileride tekrar kullanılmak üzere **joblib** ile kaydedilir.


In [ ]:

os.makedirs("models", exist_ok=True)
joblib.dump(best_logreg, "models/kdd99_logreg_pipeline.joblib")
joblib.dump(best_rf, "models/kdd99_rf_pipeline.joblib")
print("Modeller kaydedildi: models/ klasöründe.")



## VIII. Kapanış

- Random Forest modeli çoğu durumda Logistic Regresyon'a göre daha yüksek performans göstermiştir.  
- Özellik önemleri incelendiğinde ağ trafiğine ilişkin belirli metriklerin anomalileri ayırt etmede kritik olduğu gözlenmiştir.  
- Gelecek çalışmalar için: daha güncel veri setleri, gerçek zamanlı tespit senaryoları ve derin öğrenme tabanlı yöntemler önerilebilir.
